In [43]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import networkx as nx
import json
import sys
import os
from networkx.readwrite import json_graph
sys.path.insert(0, os.path.abspath('..'))
import  parser.matcher as matcher

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
graph = nx.DiGraph()

In [45]:
def load_sigma_file(filename):
    with open(filename, 'r') as jsonfile:
        graph_json = json.load(jsonfile)

    nodes = graph_json['nodes']
    edges = graph_json['edges']
    node_indices = {nodes[i]['id']: i for i in range(len(nodes))}
    for edge in edges:
        edge['source'] = node_indices[edge['source']]
        edge['target'] = node_indices[edge['target']]

    graph = json_graph.node_link_graph({'nodes': nodes, 'links': edges}, directed=True, multigraph=False)
    return nodes, graph

In [46]:
filename = '/media/sf_VBox_Shared/CaseLaw/graphs/wgaans_sigma_v2.json.json'
nodes, graph = load_sigma_file(filename)

In [47]:
graph.size()

110

In [48]:
for node in nodes:
    article_str = ', '.join(node['articles'].keys())
    if article_str == '':
        article_str = 'none'
    node['article'] = article_str

In [49]:
nodes[0]

{'abstract': 'Uitleg dekkingsomvang AVB-polis. Aansprakelijkheid werkgever bij verkeersongeval werknemer op grond van art. 7:611 BW (verzekeringsplicht niet nagekomen). Is schade werknemer letselschade in zin van polis? Functie AVB-polis en daarop gebaseerde verwachtingen rechtvaardigen ruime uitleg dekkingsomvang; dekking tegen aansprakelijkheid voor letselschade omvat in beginsel ook dekking tegen op art. 7:611 gebaseerde aansprakelijkheid.',
 'article': '7:611 bw',
 'articles': {'7:611 bw': 1},
 'count_annotation': 2,
 'count_version': 8,
 'creator': 'http://standaarden.overheid.nl/owms/terms/Hoge_Raad_der_Nederlanden',
 'date': '2012-03-30',
 'hasVersion': '',
 'id': 'http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2012:BV1295',
 'subject': 'http://psi.rechtspraak.nl/rechtsgebied#civielRecht',
 'title': 'ECLI:NL:HR:2012:BV1295 Hoge Raad , 30-03-2012 / 11/01010'}

In [50]:
hubs, authorities = nx.hits(graph)
statistics = {
    'degree' : nx.degree(graph),
    'in_degree' : graph.in_degree(),
    'out_degree' : graph.out_degree(),

    'degree_centrality' : nx.degree_centrality(graph),
    'in_degree_centrality' : nx.in_degree_centrality(graph),
    'out_degree_centrality' : nx.out_degree_centrality(graph),
    'betweenness_centrality' : nx.betweenness_centrality(graph),
    'hubs': hubs,
    'authorities': authorities
}

In [51]:
result = []
statistics_variables = [ 'count_version', 'count_annotation', 'article', 'date', 'year', 'subject', 'creator',
                        'degree', 'in_degree', 'out_degree', 
                        'degree_centrality','in_degree_centrality', 'out_degree_centrality', 'betweenness_centrality',
                        'hubs', 'authorities'
                      ]

for node in nodes:
    nodeid = node['id']
    
    atts = {var: statistics[var][nodeid] for var in statistics.keys()}
    atts['date'] = node.get('date', '')
    atts['year'] = int(nodeid.split('=')[-1].split(':')[3])
    atts['subject'] = node['subject']
    atts['creator'] = node['creator']
    atts['article'] = node['article']
    atts['count_version'] = node['count_version']
    atts['count_annotation'] = node['count_annotation']
    
    atts['id'] = nodeid.split('=')[-1]
    result.append(atts)

In [52]:
import pandas as pd
result_df = pd.DataFrame(result)
result_df = result_df.set_index('id')
result_df = result_df[statistics_variables]
result_df.head()

,count_version,count_annotation,article,date,year,subject,creator,degree,in_degree,out_degree,degree_centrality,in_degree_centrality,out_degree_centrality,betweenness_centrality,hubs,authorities
id,,,,,,,,,,,,,,,,
ECLI:NL:HR:2012:BV1295,8,2,7:611 bw,2012-03-30,2012,http://psi.rechtspraak.nl/rechtsgebied#civielR...,http://standaarden.overheid.nl/owms/terms/Hoge...,1,0,1,0.009009,0.0,0.009009,0.0,6.404323e-84,0.0
ECLI:NL:HR:2005:AT8782,5,2,none,2005-11-25,2005,http://psi.rechtspraak.nl/rechtsgebied#civielR...,http://standaarden.overheid.nl/owms/terms/Hoge...,2,0,2,0.018018,0.0,0.018018,0.0,5.712847e-40,0.0
ECLI:NL:HR:2007:BA7217,6,2,none,2007-07-13,2007,http://psi.rechtspraak.nl/rechtsgebied#civielR...,http://standaarden.overheid.nl/owms/terms/Hoge...,1,0,1,0.009009,0.0,0.009009,0.0,6.404323e-84,0.0
ECLI:NL:HR:2014:3519,8,3,"7:658 bw, 6:74 bw",2014-12-05,2014,http://psi.rechtspraak.nl/rechtsgebied#civielR...,http://standaarden.overheid.nl/owms/terms/Hoge...,2,0,2,0.018018,0.0,0.018018,0.0,6.048712e-62,0.0
ECLI:NL:HR:2010:BN9977,9,1,7:658 bw,2010-11-26,2010,http://psi.rechtspraak.nl/rechtsgebied#civielR...,http://standaarden.overheid.nl/owms/terms/Hoge...,2,0,2,0.018018,0.0,0.018018,0.0,2.648397e-04,0.0


In [53]:
outfile = '/media/sf_VBox_Shared/CaseLaw/spot/wgaans.csv'
result_df.to_csv(outfile)

In [54]:
outfile = '/media/sf_VBox_Shared/CaseLaw/spot/wgaans.json'
with open(outfile, 'w') as fp:
    json.dump(result, fp)